In [4]:
#Loading the MobileNet Model from Keras Library
from keras.applications import resnet50

# MobileNet Model takes input images of 224x224 pixels
row, col= 224, 224 

# Re-loading the MobileNet model without the top  layers
resnet = resnet50.ResNet50(weights = 'imagenet', 
                 include_top = False, 
                 input_shape =( row, col, 3))

#Freezing the layers. By default, they are trainable
for layer in resnet .layers:
    layer.trainable = False

# Printing the details of the models's layers 
print("*****************************")
print("List of all the Layers in the model : \n")
for (n,layer) in enumerate(MobileNet.layers):
    print(str(n) + " : "+ layer.__class__.__name__, layer.trainable)

#This function is to create the Top Layer for the model which will be placed above the bottom layers
def create_head(bottom_layers, num_classes):
    top_layer = bottom_layers.output
    top_layer = GlobalAveragePooling2D()(top_layer)
    top_layer = Dense(1024,activation='relu')(top_layer)
    top_layer = Dense(1024,activation='relu')(top_layer)
    top_layer = Dense(512,activation='relu')(top_layer)
    top_layer = Dense(num_classes,activation='softmax')(top_layer)
    return top_layer

#Placing the Top layer above the Bottom Layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 4
num_classes = 5
FC_Head = create_head(resnet , num_classes)
model = Model(inputs = resnet.input, outputs = FC_Head)
print("*****************************")
print("Model's Summary : \n")
print(model.summary())

### Loading our Dataset containing face Images
from keras.preprocessing.image import ImageDataGenerator

#**************************************
train_data_dir = 'Dataset2/train/'
validation_data_dir = 'Dataset2/validation/'
#**************************************

#AUGMENTATION 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Specify Batch Size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(row,col),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(row, col),
        batch_size=batch_size,
        class_mode='categorical')

### Training out Model
#Note we're using checkpointing and early stopping

from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
             
checkpoint = ModelCheckpoint("face_recognition_model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 1, restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',optimizer = RMSprop(lr = 0.001), metrics = ['accuracy'])


E:\SOFTWARES\AnacondaInstall\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


*****************************
List of all the Layers in the model : 

0 : InputLayer False
1 : ZeroPadding2D False
2 : Conv2D False
3 : BatchNormalization False
4 : Activation False
5 : ZeroPadding2D False
6 : MaxPooling2D False
7 : Conv2D False
8 : BatchNormalization False
9 : Activation False
10 : Conv2D False
11 : BatchNormalization False
12 : Activation False
13 : Conv2D False
14 : Conv2D False
15 : BatchNormalization False
16 : BatchNormalization False
17 : Add False
18 : Activation False
19 : Conv2D False
20 : BatchNormalization False
21 : Activation False
22 : Conv2D False
23 : BatchNormalization False
24 : Activation False
25 : Conv2D False
26 : BatchNormalization False
27 : Add False
28 : Activation False
29 : Conv2D False
30 : BatchNormalization False
31 : Activation False
32 : Conv2D False
33 : BatchNormalization False
34 : Activation False
35 : Conv2D False
36 : BatchNormalization False
37 : Add False
38 : Activation False
39 : Conv2D False
40 : BatchNormalization False
41 

Found 25 images belonging to 5 classes.


In [2]:
# Enter the number of training and validation samples here
#**************************************
nb_train_samples = 93
nb_validation_samples = 25
#**************************************

# We train 10 EPOCHS because out dataset is small
epochs = 3
batch_size = 16

print("*****************************")
print("EPOCHS : ")
history = model.fit_generator(train_generator, steps_per_epoch = nb_train_samples // batch_size, epochs = epochs, callbacks = callbacks, validation_data = validation_generator, validation_steps = nb_validation_samples // batch_size)

*****************************
EPOCHS : 

Epoch 1/3
5/5 [==============================] - 128s 26s/step - loss: 8.0454 - accuracy: 0.1720 - val_loss: 1.6767 - val_accuracy: 0.2000

Epoch 00001: val_loss improved from inf to 1.67669, saving model to face_recognition_model.h5
Epoch 2/3
5/5 [==============================] - 128s 26s/step - loss: 1.7038 - accuracy: 0.2914 - val_loss: 1.7189 - val_accuracy: 0.2000

Epoch 00002: val_loss did not improve from 1.67669
Epoch 3/3
5/5 [==============================] - 114s 23s/step - loss: 1.2909 - accuracy: 0.4904 - val_loss: 1.7001 - val_accuracy: 0.2000

Epoch 00003: val_loss did not improve from 1.67669


In [3]:
# Loading our model
from keras.models import load_model
classifier = load_model('face_recognition_model.h5')

#Importing Required Libraries
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join


face_recog_dict = {"[0]": "ben_afflek ", 
                      "[1]": "elton_john",
                      "[2]": "jerry_seinfeld",
                     "[3]" :"madonna",
                     "[4]":"mindy_kaling", 
                     }
face_recog_dict_n = {"ben_afflek": "ben_afflek ", 
                      "elton_john": "elton_john",
                      "jerry_seinfeld": "jerry_seinfeld",
                     "madonna" :"madonna",
                     "mindy_kaling":"mindy_kaling",                     
                     }


def draw_test(name, pred, im):
    face = face_recog_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)


def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_recog_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    


for i in range(0,10):
    input_im = getRandomImage("Dataset2/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)


cv2.destroyAllWindows()

Class - elton_john
Class - mindy_kaling
Class - mindy_kaling
Class - madonna
Class - madonna
Class - madonna
Class - mindy_kaling
Class - elton_john
Class - madonna
Class - madonna
